In [1]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError,ChunkedEncodingError
import itertools
import multiprocessing
import Ipynb_importer
from IPpool import GetHeaders
from IPpool import Ip_pool
import random
import copy
import os
# import requests_toolbelt.adapters.appengine

# requests_toolbelt.adapters.appengine.monkeypatch()


importing Jupyter notebook from IPpool.ipynb


In [2]:
def requests_visit_url_test(url,is_use_ip =False,timeout=5):
        global ip_pool
        global bad_ip
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {
                    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                    'Accept-Encoding':'gzip, deflate',
                    'Accept-Language':'zh-CN,zh;q=0.8,en;q=0.6',
                    'Cache-Control':'max-age=0',
                    'Connection':'keep-alive',
                    #'Cookie':'_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; __mta=222713764.1505836341840.1507819135553.1507819166825.4; wed_user_path=163|0; Hm_lvt_dbeeb675516927da776beeb1d9802bd4=1508317369; __utma=1.1801105885.1505634423.1505636820.1508318009.3; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; JSESSIONID=E0D0810114F31617ED97BC416CFDCE80; aburl=1; cy=1; cye=shanghai; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; s_ViewType=10; _lxsdk_s=15f304e1839-37c-2e4-6cf%7C%7C174',                    'Host':'www.dianping.com',
                    #'If-Modified-Since':'Wed, 18 Oct 2017 08:58:29 GMT',
                    #'Referer':'http://www.dianping.com/search/category/219/30/g141',
                    #'Upgrade-Insecure-Requests':'1',
                    'User-Agent':random.choice(GetHeaders().user_agent_list)}

        #获取任意一个请求头
        #headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        print(headers)
        bad_ip = []  #用来装坏的ip
        
        #*********如果不设置ip的话**********************************
        if is_use_ip == False:
            
            res = requests.get(url,headers = headers,timeout=timeout)  #访问url，不设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            if soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                print("ip被禁止了！！！！！！！！！！！！！！！！")
                raise ConnectionError
           
        #************如果设置ip的话，则执行这一段*********************
        else:
            is_continue = True   #设置循环标志
            while is_continue:
            #整理成ip地址的格式
                try:
                    #
                    print(len(bad_ip))
                    if len(ip_pool)==len(bad_ip):
                        ip_url = ['http://www.xicidaili.com/nn/']
                        test_url = 'http://www.dianping.com/'
                        ip_pool= Ip_pool().get_ip_pool(test_url,timeout=2)
                        prox_ip = random.choice(ip_pool)
                        bad_ip = []
                        
                    #******随机生成一个ip*******
                    prox_ip = random.choice(ip_pool)
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    
                    #*******用代理ip访问********
                    res = requests.get(url,headers = headers,proxies = proxies,timeout=timeout) #设代理ip访问
                    html =  res.content.decode('utf-8')
                    soup  =  BeautifulSoup(html,'lxml')
                    
                    
                    #********如果返回网页被禁止的情况，触发异常,否则返回正常**********
                    if  soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                        print("ip被禁止了!!!!!!!!!!!!!!!!!!!!")
                        raise ConnectionError
                    else:
                        is_continue = False

                except HTTPError as e:
                    bad_ip.append(prox_ip)
                    print("断网")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(10)


                except Timeout as e:
                    bad_ip.append(prox_ip)
                    print("超时")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(10)


                except ConnectionError as e:
                    bad_ip.append(prox_ip)
                    print("访问被拒")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(10)
                    
                except AttributeError as e:
                    bad_ip.append(prox_ip)
                    print("----------",soup)
                    prox_ip = random.choice(ip_pool)
                    time.sleep(10)
            

        return soup

In [3]:
#**************将装机地址变成一组一组*****************
def get_chunk(adr_ls,n):
    new_adr_ls = copy.deepcopy(adr_ls)
    adr_chunk = [new_adr_ls[i:i + n] for i in range(0, len(new_adr_ls), n)]
    return adr_chunk

In [4]:

#设置谷歌浏览器的参数
def set_chrome_param():
    #PROXY = '175.155.24.61:808'
    options = webdriver.ChromeOptions()
    chrome_path = '/home/zhh/下载/chromedriver'  
    #options.add_argument('--proxy-server={0}'.format(PROXY))
    # 设置成中文
    options.add_argument('lang=zh_CN.UTF-8')
    # 添加头部
    #1允许所有图片；2阻止所有图片；3阻止第三方服务器图片
    prefs = {'profile.default_content_setting_values': { 'images':2}}
    options.add_experimental_option('prefs', prefs)
    options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"')
    chrome = webdriver.Chrome(chrome_path,chrome_options=options)
    return chrome


In [5]:
#requests请求网页，返回一个BeautifulSoup对象
def requests_visit_url(url,prox_ip=None):
    
    #*******设置请求头*************
    headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":url,
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}
    headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
    #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"}
    
    #********是否循环*************
    is_continue = True
    num = 0
    while is_continue:
        num +=1
        print("访问{0}次".format(num))
        try:
            #***********不设代理访问*************
            if prox_ip is None:
                res = requests.get(url,headers = headers)  #访问url，不设代理ip访问
            else:
                
            #*********设置代理ip访问*************
                http = prox_ip[0]
                ip = http+"://"+prox_ip[1]
                proxies={http:ip}
                print(proxies)
                res = requests.get(url,headers = headers,proxies = proxies) #设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            is_continue = False 
            
        except HTTPError as e:
            print("断网")
            time.sleep(2)
            
        except Timeout as e:
            print("请求超时")
            time.sleep(2)
            
        except ConnectionError as e:
            print("访问被拒")
            is_continue = True
            time.sleep(2)
    return soup

In [6]:
#缓存结果
def save_cache(result,filename):
    with open(filename,'wb') as f1:
        pickle.dump(result,f1)

In [7]:
#加载缓存
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

In [8]:
# 切换路径
def get_spc_path(father_path,child_path):
    os.chdir(father_path)
    if os.path.exists(child_path):
        os.chdir(child_path)
    else:
        os.mkdir(child_path)
        os.chdir(child_path)
    print("done!!")

## 1.二级分类层

In [9]:
%%time
#获取所有的二级分类url
def classify(city_url,city,big_class):
    soup = requests_visit_url(city_url)
    clf_ls = []
    for i in soup.select("#classfy a"):
        clf_url = i["href"] #分类网址
        clf_tl = i.text.strip() # 分类标题
        clf_ls.append([big_class,city,clf_tl,clf_url])
    return clf_ls

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.9 µs


In [11]:
# %%time
# clf_ls = classify("http://www.dianping.com/search/category/7/10",'深圳',"美食")

## 2.区县层

In [10]:
def get_region(class_ls=None,prox_ip=None):
    print('正在爬取{0}'.format(class_ls[-2]))
    url = class_ls[-1]  #url
    soup = requests_visit_url(url,prox_ip) #访问
    region_ls = [] #地区的列表
    for i in soup.select("#region-nav a"):
        if i.text.strip() != "不限":
            region_url = i["href"] 
            region_tl = i.text.strip()
            region = class_ls[:-1]
            region.extend([region_tl,region_url])
            region_ls.append(region)
    return region_ls


In [11]:
# ip = Ip_pool(['http://www.xicidaili.com/nn/'])
# ip_pool = ip.get_ip_pool(test_url='http://www.dianping.com/search/category/7/10/g104',timeout =1)

In [12]:
# %%time
# prox_ip = random.choice(ip_pool)
# region_ls = []
# for i in clf_ls:
#     print(i)
#     sub_region = get_region(i,prox_ip)
#     region_ls.append(sub_region)

In [13]:
# region_ls

## 3.商圈层

In [11]:
def get_circle(region_ls=None):
    url = region_ls[-1]   #url网址
    soup = requests_visit_url(url) #访问BeautifulSoup
    circle_ls =[]             
    for i in soup.select("#region-nav-sub a"):  #商圈信息
        if i.span.text.strip()!="不限":   #剔除掉不限的url
            circle_url = i["href"]       #商圈url
            circle_tit = i.span.text.strip()  #商圈名
            circle = region_ls[:-1] 
            circle.extend([circle_tit,circle_url]) 
            circle_ls.append(circle)
            
    #如果没有商圈的话，就拿原来的url当作是商圈的url
    if circle_ls==[]:
        print(region_ls[-2]+"没有子分类！！")
        region_ls.insert(4,"没有商圈")
        circle_ls.append(region_ls)

    return circle_ls
    

In [15]:
# %%time
# circle_ls = get_circle(['美食',
#   '广州',
#   '茶餐厅',
#   '天河区',
#   'http://www.dianping.com/search/category/4/10/g207r22'])

## 4.商家信息层

In [12]:
#获取每个商家的链接、店名
def get_stores(circle_ls = None):
    store_ls=[] #存储的列表
    url = circle_ls[-1]  #url
    soup = requests_visit_url_test(url,is_use_ip=True,timeout=3)  #访问当前主页
    is_continue = True
    page = 0
    while is_continue:
        page += 1
        print("正在爬取第{0}页".format(page))
        
        #解析数据
        if soup.select(".content") ==[]:
            print("此页没有任何信息！！")
            store_ls = []
            is_continue = False 
            
        else:
            for i in soup.select(".content"):  
                for j in i.select(".pic"):
                    store_name = j.img["alt"]   #店名
                    link = j.a["href"]     #详情层
                    store = circle_ls[:5]  #只取前5位
                    store.extend([url,store_name,link]) 
                    store_ls.append(store)


            #获取下一页的数据

            next_tag = soup.find_all("a",text = re.compile("下一页"))  #获取字符属性为下一页的url
            if next_tag != []:                                         #保证含有字符属性
                url = next_tag[0]["href"]                              #获取下一页的url
                soup = requests_visit_url_test(url,is_use_ip=True,timeout=2)   #更新BeautifulSoup对象
            else:
                url = ''
                is_continue  = False                                  #如果没有下一页，就停止更新
    
    return store_ls

In [17]:
# %%time
# store_ls = get_stores(['美食',
#   '广州',
#   '茶餐厅',
#   '天河区',
#   '天河城/体育中心',
#   'http://www.dianping.com/search/category/4/10/g207r1519'])


## 5.详情层

In [13]:
#获取详情层的数据，如星级用户、浏览人数、均价
def get_info(store_ls=None):
    
    #首先先保证导入的数据不为空
    if store_ls !=[]:
        url = store_ls[-1] #详情信息表
        soup = requests_visit_url_test(url,is_use_ip =True,timeout=2)
        try:
            start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
        except Exception as e:
            print("start",e)
            start = ''

        try:
            review_count = soup.select("#reviewCount")[0].text  #浏览人数
        except Exception as e:
            review_count = ''
            print("review_coun",e)

        try:
            avgprice = soup.select("#avgPriceTitle")[0].text   #均价
        except Exception as e:
            avgprice = ''
            print("avgprice",e)

        #评分系统
        try:
            score= []
            for j in soup.select("#comment_score .item"):
                score.append(j.text)
            score = "|".join(score)

        except Exception as e:
            score = ""
            print("score",e)

        try:   
            address = soup.select(".address .item")[0].text.strip() #地址
        except Exception as e:
            address = ''
            print("address出错了",e)

        #电话号码
        try:
            tel =[]
            for i in soup.select(".tel .item"):
                tel.append(i.text)
            tel = "|".join(tel)

        except:
            tel=""
            print("tel出错了",e)

        #路径
        try:
            path = []
            for i in soup.select(".breadcrumb a"):
                path.append(i.text.strip())
            path ='>'.join(path)
        except Exception as e:
            print("path出错了",e)
            path =''
            
        #五星评论人数
        try:
            
            five_starts = soup.find_all(id=re.compile("reviewCountStar\d"))
            if five_starts ==[]:
                five =''
                four =''
                three =''
                two =''
                one =''
            else:
                five,four,three,two,one = list(map(lambda x:x.text.strip(),five_starts))
        except Exception as e:
            print("五星评论人数出错","---->",e)
            five =''
            four =''
            three =''
            two =''
            one =''

        info = store_ls[:]
        info.extend([start,review_count,avgprice,score,address,tel,five,four,three,two,one,path])
        time.sleep(1)
    return info


In [19]:
# get_info(['美食',
#   '广州',
#   '茶餐厅',
#   '天河区',
#   '天河城/体育中心',
#   'http://www.dianping.com/search/category/4/10/g207r1519p2',
#   '粤港烧卤鸭饭(天河店)',
#   'http://www.dianping.com/shop/94311152'])

## 6 主程序

In [14]:
%%time

clf_ls = classify("http://www.dianping.com/search/category/7/10/r1556",'深圳',"美食")
print("所有二级分类已经获取完毕")

region_ls = list(map(get_region,clf_ls))
print("获取所有的区域清单") 

访问1次
所有二级分类已经获取完毕
正在爬取面包甜点
访问1次
正在爬取小吃快餐
访问1次
正在爬取粤菜
访问1次
正在爬取自助餐
访问1次
正在爬取咖啡厅
访问1次
正在爬取日本料理
访问1次
正在爬取火锅
访问1次
正在爬取西餐
访问1次
正在爬取小龙虾
访问1次
正在爬取海鲜
访问1次
正在爬取烧烤
访问1次
正在爬取韩国料理
访问1次
正在爬取川菜
访问1次
正在爬取东南亚菜
访问1次
正在爬取湘菜
访问1次
正在爬取茶餐厅
访问1次
正在爬取素菜
访问1次
正在爬取江浙菜
访问1次
正在爬取东北菜
访问1次
正在爬取创意菜
访问1次
正在爬取新疆菜
访问1次
正在爬取西北菜
访问1次
正在爬取粥粉面
访问1次
正在爬取私房菜
访问1次
正在爬取台湾菜
访问1次
正在爬取农家菜
访问1次
正在爬取其他
访问1次
正在爬取家常菜
访问1次
正在爬取下午茶
访问1次
获取所有的区域清单
CPU times: user 6.63 s, sys: 112 ms, total: 6.74 s
Wall time: 36.8 s


In [15]:
#解链
region_ls = list(itertools.chain.from_iterable(region_ls))
region_ls

[['美食',
  '深圳',
  '面包甜点',
  '福田区',
  'http://www.dianping.com/search/category/7/10/g117r29'],
 ['美食',
  '深圳',
  '面包甜点',
  '南山区',
  'http://www.dianping.com/search/category/7/10/g117r31'],
 ['美食',
  '深圳',
  '面包甜点',
  '罗湖区',
  'http://www.dianping.com/search/category/7/10/g117r30'],
 ['美食',
  '深圳',
  '面包甜点',
  '宝安区',
  'http://www.dianping.com/search/category/7/10/g117r33'],
 ['美食',
  '深圳',
  '面包甜点',
  '龙岗区',
  'http://www.dianping.com/search/category/7/10/g117r34'],
 ['美食',
  '深圳',
  '面包甜点',
  '龙华新区',
  'http://www.dianping.com/search/category/7/10/g117r12033'],
 ['美食',
  '深圳',
  '面包甜点',
  '盐田区',
  'http://www.dianping.com/search/category/7/10/g117r32'],
 ['美食',
  '深圳',
  '面包甜点',
  '大鹏新区',
  'http://www.dianping.com/search/category/7/10/g117r12036'],
 ['美食',
  '深圳',
  '面包甜点',
  '光明新区',
  'http://www.dianping.com/search/category/7/10/g117r12034'],
 ['美食',
  '深圳',
  '面包甜点',
  '坪山新区',
  'http://www.dianping.com/search/category/7/10/g117r12035'],
 ['美食',
  '深圳',
  '小吃快餐',
  '福田区',
  'http:/

In [16]:
%%time
#***************获取所有的商圈信息****************************888
circle_ls = list(map(get_circle,region_ls))

访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
光明新区没有子分类！！
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1次
访问1

In [17]:
#解链
circle_ls = list(itertools.chain.from_iterable(circle_ls))

In [18]:
get_spc_path("/home/zhh/dzdp_data/","szmeishi")

done!!


In [19]:
save_cache(circle_ls,"/home/zhh/dzdp_data/szmeishi/深圳_circle_ls.pickle")

In [20]:
circle_ls = read_cache("/home/zhh/dzdp_data/szmeishi/深圳_circle_ls.pickle")
print(len(circle_ls))
circle_chunk = get_chunk(circle_ls,5)
len(circle_chunk)

2342


469

In [22]:
offset=0
save_cache(offset,"/home/zhh/dzdp_data/szmeishi/深圳_offset.pickle")

In [ ]:
%%time
if __name__=='__main__':
    
    
    is_continue = True
    while is_continue:
        
        try:
            #如果偏移量已经达到了660,跳出整个大循环
            offset = read_cache("/home/zhh/dzdp_data/szmeishi/深圳_offset.pickle")
            end = len(circle_chunk)
            if offset >= len(circle_chunk):
                print("整个程序已经完毕！！！")
                break
                
            #************爬取代理ip******************
            global ip_pool
            ip_url = ['http://www.xicidaili.com/nn/']
            test_url = "http://www.dianping.com/shop/2954893"
            ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)
        
                
            #***********多进程处理*************************
            for index,chunk in enumerate(circle_chunk[offset:end]):
                offset_index = offset + index                       #偏移量+计数器
                print("正在爬第{0}chunk".format(offset_index))
                pool = multiprocessing.Pool(processes=3)           # 创建3个进程
                result = []
                for i in chunk:
                    result.append(pool.apply_async(get_stores, (i, ))) #将参数全部加进去
                pool.close()                                  # 关闭进程池，表示不能再往进程池中添加进程，需要在join之前调用
                pool.join()                                   # 等待进程池中的所有进程执行完毕
                
                #***********遍历每个进程的结果*****************
                store_ls = []
                for res in result:
                    store_ls.append(res.get())               #
                
                #***********获取每一个chunk的内容****************
                print("第{0}chunk已经爬完了".format(offset_index))
                store_ls = list(itertools.chain.from_iterable(store_ls))
                save_cache(store_ls,"/home/zhh/dzdp_data/szmeishi/深圳_store_ls_{0}_chunk.pickle".format(offset_index))
                
                #********爬完直接将偏移量设为下一个*************************
                last_offset_1 = offset_index+1
                save_cache(last_offset_1,"/home/zhh/dzdp_data/szmeishi/深圳_offset.pickle")
                
        #***************如果出现ChunkedEncodingError重复执行这段代码*********************     
        except ChunkedEncodingError as e:
            print("出现ChunkedEncodingError")
            last_offset_2 = offset_index
            print("chunkedEncodingError保存offset:{0}".format(last_offset_2))
            save_cache(last_offset_2,"/home/zhh/dzdp_data/szmeishi/深圳_offset.pickle")
            continue
            
        #***************如果出现ChunkedEncodingError重复执行这段代码*********************      
        except KeyboardInterrupt as e:
            print("手动停止")
            last_offset_3 = offset_index
            print("KeyboardInterrupt保存现有的offset:{0}".format(last_offset_3))
            save_cache(last_offset_3,"/home/zhh/dzdp_data/szmeishi/深圳_offset.pickle")
            break
            
            
#     print("***********准备爬商家的详情信息******************")
#     pool = multiprocessing.Pool(processes=3) # 创建3个进程
#     result=[]
#     for i in store_ls:
#         result.append(pool.apply_async(get_info,(i,)))
#     pool.close()
#     pool.join()
    
#     print("信息页已经爬完了！！")
#     info_ls=[]
#     for res in result:
#         info_ls.append(res.get())
#     save_cache(info_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_info_test_ls.pickle")

一共获取了36个ip
正在爬第0chunk
{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'zh-CN,zh;q=0.8,en;q=0.6', 'Cache-Control': 'max-age=0', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3'}
0
{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'zh-CN,zh;q=0.8,en;q=0.6', 'Cache-Control': 'max-age=0', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5'}
{'http': 'http://221.10.159.234:1337'}
0
{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'zh-CN,zh;q=0.8,en;q=0.6', 'Cache-Control': 'max-

## 爬取所有的商家信息

In [21]:
def merge_chunk(file_dir,regrex="(.*?_store_ls_\d+_chunk.pickle)"):
    file_ls = os.listdir(file_dir)  #得出列出文件
    re_ptn = re.compile(regrex)   #正则表达式
    files = []
    result =[]
    for i in file_ls:
        if re_ptn.search(i):
            files.append(i)
    
    print("合并{0}个chunk".format(len(files)))
    for i in files:
        sub_result = read_cache(file_dir+i)
        result.extend(sub_result)
    return result

##### 佛山

In [22]:
result = merge_chunk("/home/zhh/dzdp_data/fsmeishi/")

合并214个chunk


In [23]:
info_chunk = get_chunk(result,10)
len(info_chunk)

3110

In [68]:
global ip_pool
ip_url = ['http://www.xicidaili.com/nn/']
test_url = "http://www.dianping.com/shop/9310903"
ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)

http://www.xicidaili.com/nn/
{'Accept': 'application/json, text/javascript, */*; q=0.01', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'zh-CN,zh;q=0.8', 'Connection': 'keep-alive', 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1'}
坏ip: ['http', 'http://119.55.48.225:9999']
坏ip: ['http', 'http://110.87.3.211:43474']
坏ip: ['http', 'http://121.31.79.43:8123']
坏ip: ['http', 'http://61.178.238.122:63000']
坏ip: ['http', 'http://222.189.110.184:21558']
坏ip: ['http', 'http://111.155.116.224:8123']
坏ip: ['http', 'http://121.31.196.177:8123']
坏ip: ['http', 'http://113.120.61.96:33805']
坏ip: ['http', 'http://110.73.54.1:8123']
坏ip: ['http', 'http://111.155.116.217:8123']
坏ip: ['http', 'http://121.31.100.95:8123']
坏ip: ['http', 'http://110.73.7.252:8123']
坏ip: ['http', 'http://49.87.146.96:45340']
坏ip: ['http', 'http://110.72.19.194:8123']
坏ip: 

In [69]:
ip_pool

[['https', 'https://123.185.129.5:8118'],
 ['https', 'https://119.55.217.251:9999'],
 ['https', 'https://180.173.82.97:8118'],
 ['http', 'http://111.200.58.94:80'],
 ['http', 'http://118.114.77.47:8080'],
 ['https', 'https://115.206.172.95:1080'],
 ['https', 'https://218.15.25.153:808'],
 ['https', 'https://112.86.112.64:9999'],
 ['https', 'https://153.34.199.194:80'],
 ['https', 'https://183.63.101.62:53281'],
 ['https', 'https://171.108.205.85:53281'],
 ['https', 'https://180.115.10.141:35211'],
 ['https', 'https://115.200.30.1:80'],
 ['https', 'https://111.78.172.92:22467'],
 ['https', 'https://110.73.7.159:8123'],
 ['https', 'https://120.78.15.63:80'],
 ['https', 'https://119.55.116.118:9999'],
 ['https', 'https://113.128.29.31:43319'],
 ['https', 'https://36.35.127.240:9999'],
 ['https', 'https://117.78.51.231:3128'],
 ['https', 'https://120.40.134.218:27540'],
 ['https', 'https://218.56.200.156:8118'],
 ['https', 'https://112.254.122.59:8118'],
 ['https', 'https://113.236.174.213

In [81]:
info_chunk[0][0]

['美食',
 '佛山',
 '西北菜',
 '南海区',
 '保利水城',
 'http://www.dianping.com/search/category/208/10/g26481r66732',
 '九毛九山西面馆(保利水城店)',
 'http://www.dianping.com/shop/22405680']

In [99]:
headers = {'User-Agent':random.choice(GetHeaders().user_agent_list)}
print(headers)
requests.get("http://www.dianping.com/shop/22405680",proxies = {"http":"http://139.196.12.124:8088"}
             ,headers=headers)

{'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3'}


ConnectionError: ('Connection aborted.', BadStatusLine('<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\r\n',))

In [97]:
requests_visit_url_test("http://www.dianping.com/shop/22405680",is_use_ip=True)

{'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'Accept-Encoding': 'gzip, deflate', 'Accept-Language': 'zh-CN,zh;q=0.8,en;q=0.6', 'Cache-Control': 'max-age=0', 'Connection': 'keep-alive', 'Upgrade-Insecure-Requests': '1', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6'}
0
{'https': 'https://110.85.132.65:8118'}
ip被禁止了!!!!!!!!!!!!!!!!!!!!
访问被拒
1
{'https': 'https://119.55.116.118:9999'}
ip被禁止了!!!!!!!!!!!!!!!!!!!!
访问被拒
2
{'https': 'https://180.173.82.97:8118'}
ip被禁止了!!!!!!!!!!!!!!!!!!!!
访问被拒


KeyboardInterrupt: 

In [ ]:
Accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
Accept-Encoding:gzip, deflate
Accept-Language:zh-CN,zh;q=0.9
Cache-Control:max-age=0
Connection:keep-alive
Cookie:_hc.v=9a05f229-418f-ebde-dece-2fb8c63fe38b.1508380216; _lxsdk_cuid=15f32812866c8-06022b9b7afe8a-102c1709-100200-15f32812866c8; _lxsdk=15f32812866c8-06022b9b7afe8a-102c1709-100200-15f32812866c8; s_ViewType=1; JSESSIONID=D72E6CE714C0447A298A923E8CB451C1; aburl=1; cy=4; cye=guangzhou; __mta=122090932.1508380981570.1508380981570.1508403803120.2; _lxsdk_s=15f33de7ac9-020-037-894%7C%7C18
Host:www.dianping.com
If-Modified-Since:Wed, 18 Oct 2017 08:47:09 GMT
Upgrade-Insecure-Requests:1
User-Agent:Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36